# Requirements

In [1]:
!pip install javalang

In [49]:
import os
import hashlib
import ast       #Library to produce AST and analyze it
import astor     #Library to produce the actual code of an AST
import pandas as pd
import numpy as np
import re
#import javalang
#from javalang.tree import *

In [2]:
hash_dict = {}
filesname = []
file_directories = []

## Projects to Investigate

Here we will investigate Apache projects which have used python in their implementation. Among 22 available python projects we have eliminated 6 projects (either because they are not active anymore or they do not have good repository). There are 9 projects among remaining 16 that are well defined and maintained. We will be keep working on these projects to eventualy come up with our case studies, so that we can start testing our model then.

In [3]:
"""
In this part we make a list of the files' name and their own directory in the given directory.
A dictionary is also produced having a hash value of the files name as the key and their actual name as the value.
"""

#directory = "/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate"
directory = "/home/amirmahdi/projects/Adaptive-Logging-system-git/code/projects-to-investigate/"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        filesname.append(filename)
        file_directories.append(f)

# Extracting function blocks from source code

In [4]:
"""
This function accepts the source code as a string. It first parse it to AST, then iterate through AST and selects 
nodes that are representing function definition (FunctionDef)
"""

def extract_functions(source_code):
    tree = ast.parse(source_code)
    functions = []
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            functions.append(node)
    return functions

In [5]:
"""
Creating the func_of_files list which each of its elements it is a list of function nodes of a file in the
directory.
These nodes (subtree of the AST) could be used to creat the actual functions in string by using following code:

source_code = astor.to_source(node)

Also, in this part, an ID is assigned to each function (based on its name) by using hashlib, then this ID is used as 
the key value of the "hash_dict" which holds a list containing function's name, its directory and list of
neighbour functions as the dictionary's value.
"""
#here=0
func_nodes_of_files = []
for file_dir in file_directories:
    file_name = file_dir.split('/')[-1]
    with open(file_dir, "r") as file:
        source_code = file.read()
        """here = here+1
        print(here)"""
    try:
        functions = extract_functions(source_code)
        for function in functions:
            functions_copy = functions.copy() # Make a copy of list of functions found in current file
            functions_copy.remove(function)    #creating a list of neighbours of current function (in itteratin) by removing itself from functions_copy
            hash_dict[ hashlib.sha1(str.encode(function.name + file_name)).hexdigest() ] = [file_dir.split("/")[-1], file_dir, 
                                                                               function.name, functions_copy]
            func_nodes_of_files.append([function, file_name])
        #func_nodes_of_files.extend(functions)
    except SyntaxError:
        print(SyntaxError)
        pass


<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'SyntaxError'>
<class 'Sy

In [ ]:
func_nodes_of_files[0][0].name

In [ ]:
source = astor.to_source(func_nodes_of_files[0][0])
print(source)

# Structring Dataset by extracting functions' features

## 1. Input-dependent loops

* #### Defining needed functions:

In [6]:
def loop_finder(function_node):
    loops = []
    for node in ast.walk(function_node):
        if isinstance(node, ast.For) or isinstance(node, ast.While):
            loops.append(node)
    return loops

In [7]:
"""This function takes a source code as input and finds the name of all variables which hold the result of 
a function call"""

def dependent_variable_finder(source_code):
    tree = ast.parse(source_code)             # creating source code's AST
    d_variable_names = []
    # here it funds variable holding function calls' result
    for node in ast.walk(tree):
        if isinstance(node, ast.Assign):
            for node_child in ast.walk(node):
                if isinstance(node_child, ast.Call):
                    # Parse back the variable assignment node to source code and extract variable names
                    d_variable_names.extend(astor.to_source(node).split("=")[0].split(",")) 
                    
    return d_variable_names

* #### Creating Dataframe,then adding "ID" and "number_of_loops" of a fuction to it:

In [38]:
func_nodes_of_files

[[<_ast.FunctionDef at 0x7fdbdea60e90>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbcf765a90>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbd7f49090>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbd7f49a50>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbd7f49f10>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbd7f37f10>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbcf7722d0>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbcf776390>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbcf77b490>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbcf77ff50>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbcf786410>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbd7ca4110>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbd7ca4710>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbd7ca4a10>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbd7ca4dd0>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0x7fdbd7c9f110>, 'blobstorageio.py'],
 [<_ast.FunctionDef at 0

In [8]:
df = pd.DataFrame(columns = ['ID', 'number_of_loops'])
for function_node in func_nodes_of_files:
    found_loops = loop_finder(function_node[0])
    ID = hashlib.sha1(str.encode(function_node[0].name + function_node[1])).hexdigest()
    number_of_loops = len(found_loops)
    
    new_row = pd.Series({"ID":ID, "number_of_loops":number_of_loops})
    df = pd.concat([df, new_row.to_frame().T], ignore_index=True)
df = df.set_index("ID")


* #### Adding "loop_input_dependent_level" to the dataframe:
    
    For all of the functions, first we extract all loops inside it (if there is any) by "using loop_finder" function. Then we creat a list of kewords representing input_dependent variables and neighbor functions. Doing these two steps, we then search for the keywords in loops' declaration part, and we increment the "input_dependent_level" of the given function if any keyword found.

In [9]:
"""This is a recursive function to find a nesting level of a given node representing a loop"""
def nested_loop_level_finder(loop_nodes):
    nested_loop_level = 0
    for loop in loop_nodes:
        nested_loop_level += len(loop_finder(loop))
    return nested_loop_level

In [50]:
x = ""
for function_node in func_nodes_of_files:
    """print('.............................')
    print(function_node.name)"""
    input_dependent_level = 0
    found_loops = loop_finder(function_node[0]) # here we first find loops.
    ID = hashlib.sha1(str.encode(function_node[0].name + function_node[1])).hexdigest()
    file_name = hash_dict[ID][1].split('/')[-1]
    if len(found_loops) != 0: #if any loop was found
        # use "has_dict" to get the neighbour functions of the curent function by using its ID (key):
        neighbour_functions_names = [n_function.name for n_function in hash_dict[ID][3]]
        
        if file_name != x:
            x = file_name
            with open(hash_dict[ID][1], "r") as file: # hash_dict[ID][1] gives the directory of current function's file
                source_code = file.read()
            dependent_variable_names = dependent_variable_finder(source_code)
        
        keywords = dependent_variable_names + neighbour_functions_names
        for loop in found_loops: # "loop" here is a node of a loop.
            #print(astor.to_source(loop))
            for keyword in keywords:
                pattern = fr'\b{re.escape(keyword)}\b'
                if bool(re.search(pattern, astor.to_source(loop).split("\n")[0])):
                    #print(function_node[0].name , keyword)
                    #print(astor.to_source(loop).split("\n")[0])
                    input_dependent_level += 1 
    df.at[ID, "loop_input_dependent_level"] = int(input_dependent_level)

copy_tree list_prefix
for entry in self.list_prefix(src):
delete_files container
for container, blob in zip(containers, blobs):
delete_files container
for container, blob in zip(containers, blobs):
delete_files containers
for container, blob in zip(containers, blobs):
delete_files containers
for container, blob in zip(containers, blobs):
delete_files  container
for container, blob in zip(containers, blobs):
delete_files container
for container, blobs in grouped_blobs.items():
delete_files container
for container, blobs in grouped_blobs.items():
delete_files  container
for container, blobs in grouped_blobs.items():
_delete_batch  blob 
for blob in blobs:
_delete_batch  blob 
for blob in blobs:
_delete_batch  blob 
for blob in blobs:
list_prefix list_files
for file_metadata in self.list_files(path, with_metadata):
getNonConfiguredHaHost host 
for host in response:
getNonConfiguredHaHost host 
for host in response:
getNonConfiguredHaHost host 
for host in response:
getNonConfiguredHaHost 

test_write_pipeline_non_globalwindow_input file_name 
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_non_globalwindow_input file_name 
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_non_globalwindow_input file_name 
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_non_globalwindow_input file_name 
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_non_globalwindow_input file_name 
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_non_globalwindow_input file_name 
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_non_globalwindow_input file_name 
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_non_globalwindow_input file_name 
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_non_globalwindow_input file_name 
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_non_globalwindow_input file_name 
for file_name in glob.glob(self.path + '*'):
test_write

test_write_pipeline_auto_compression_unsharded file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_auto_compression_unsharded file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_auto_compression_unsharded file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_auto_compression_unsharded file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_auto_compression_unsharded file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_auto_compression_unsharded file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_auto_compression_unsharded file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_auto_compression_unsharded file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_auto_compression_unsharded file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_auto_compression_unsharded file_name
for file_name in glob.glo

test_write_pipeline_footer file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_footer file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_footer file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_footer file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_footer file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_footer file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_footer file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_footer file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_footer file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_footer file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_footer file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipeline_footer file_name
for file_name in glob.glob(self.path + '*'):
test_write_pipel

test_write_max_records_per_shard file_name
for file_name in glob.glob(self.path + '*'):
test_write_max_records_per_shard file_name
for file_name in glob.glob(self.path + '*'):
test_write_max_records_per_shard file_name 
for file_name in glob.glob(self.path + '*'):
test_write_max_records_per_shard file_name 
for file_name in glob.glob(self.path + '*'):
test_write_max_records_per_shard file_name 
for file_name in glob.glob(self.path + '*'):
test_write_max_records_per_shard file_name 
for file_name in glob.glob(self.path + '*'):
test_write_max_records_per_shard file_name 
for file_name in glob.glob(self.path + '*'):
test_write_max_records_per_shard file_name 
for file_name in glob.glob(self.path + '*'):
test_write_max_records_per_shard file_name 
for file_name in glob.glob(self.path + '*'):
test_write_max_records_per_shard file_name 
for file_name in glob.glob(self.path + '*'):
test_write_max_records_per_shard file_name 
for file_name in glob.glob(self.path + '*'):
test_write_max_records_

...

test_shutdown_with_fast_workers future 
for future in futures:
test_shutdown_with_slow_workers future 
for future in futures:
test_worker_reuse future 
for future in futures:
test_shared_shutdown_does_nothing future 
for future in futures:
main run_tests
for count, (summary, stdout) in enumerate(pool_map(run_tests, paths)):
test_option_modifications_are_shared_between_views options 
for options in [pipeline_options, mock_options]:
test_option_modifications_are_shared_between_views options 
for options in [pipeline_options, mock_options]:
test_option_modifications_are_shared_between_views options 
for options in [pipeline_options, mock_options]:
test_option_modifications_are_shared_between_views options 
for options in [pipeline_options, mock_options]:
test_option_modifications_are_shared_between_views options 
for options in [pipeline_options, mock_options]:
test_option_modifications_are_shared_between_views options 
for options in [pipeline_options, mock_options]:
test_option_modifica

get_synthetic_sdf_step cur 
for cur in range(current_restriction.start, current_restriction.stop):
run known_args
for step_no, steps in enumerate(known_args.steps):
split stop 
for start, stop in bundle_ranges:
split stop 
for start, stop in bundle_ranges:
split stop 
for start, stop in bundle_ranges:
split stop 
for start, stop in bundle_ranges:
split split
for split in restriction.split(offsets_per_split, offsets_per_split // 2):
split split
for split in restriction.split(offsets_per_split, offsets_per_split // 2):
split stop 
for start, stop in bundle_ranges:
split stop 
for start, stop in bundle_ranges:
split stop 
for start, stop in bundle_ranges:
split stop 
for start, stop in bundle_ranges:
process cur 
for cur in range(current_restriction.start, current_restriction.stop):
merge accumulator 
for accumulator in accumulators:
merge accumulator 
for accumulator in accumulators:
merge accumulator 
for accumulator in accumulators:
merge accumulator 
for accumulator in accumulators:
m

insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets request 
for client, request in request_group.items():
insert_tablets reque

verify_success_by_list status 
for status in status_list:
_get_unique_vag_ids host 
for host in hosts:
_get_unique_vag_ids host 
for host in hosts:
_get_unique_vag_ids host 
for host in hosts:
_get_unique_vag_ids host 
for host in hosts:
_get_unique_vag_ids host 
for host in hosts:
_get_xenserver_host_iscsi_iqns host 
for host in hosts:
_get_xenserver_host_iscsi_iqns host 
for host in hosts:
_get_xenserver_host_iscsi_iqns host 
for host in hosts:
_get_xenserver_host_iscsi_iqns host 
for host in hosts:
_get_xenserver_host_iscsi_iqns host 
for host in hosts:
_verifyVag host_iscsi_iqn 
for host_iscsi_iqn in host_iscsi_iqns:
_fire_eligible_windows elems 
for w, elems in windows_to_elements.items():
_fire_eligible_windows elems 
for w, elems in windows_to_elements.items():
_parse_header_format  
for _ in range(transform_count):
_parse_header_format  
for _ in range(transform_count):
_parse_header_format  
for _ in range(transform_count):
_parse_header_format  
for _ in range(transform_count

createVlanIpRanges iprange 
for iprange in ipranges:
createVlanIpRanges iprange 
for iprange in ipranges:
configureProviders provider 
for provider in providers:
tearDown path 
for path in self._temp_files:
tearDown path 
for path in self._temp_files:
get_splits _get_split_key
for next_client_key in _get_split_key(client_scatter_keys, num_splits):
resolve_as_files chunk 
for chunk in retrieval_service.GetArtifact(beam_artifact_api_pb2.
resolve_as_files GetArtifact
for chunk in retrieval_service.GetArtifact(beam_artifact_api_pb2.
resolve_as_files GetArtifact
for chunk in retrieval_service.GetArtifact(beam_artifact_api_pb2.
offer_artifacts chunk 
for chunk in artifact_retrieval_service.GetArtifact(request.get_artifact):
offer_artifacts GetArtifact
for chunk in artifact_retrieval_service.GetArtifact(request.get_artifact):
offer_artifacts GetArtifact
for chunk in artifact_retrieval_service.GetArtifact(request.get_artifact):
store_artifact GetArtifact
for block in service.GetArtifact(beam_a

In [15]:
for function_node in func_nodes_of_files:
    input_dependent_level = 0
    nested_loop_level = 0
    found_loops = loop_finder(function_node[0]) # here we first find loops. 
    nested_loop_level = nested_loop_level_finder(found_loops)
    ID = hashlib.sha1(str.encode(function_node[0].name + function_node[1])).hexdigest()
    df.at[ID, "nested_loop_level"] = int(nested_loop_level)

* #### Final version of the dataset having all features related to loops:
    The dataset so far includes folowing features:
    * **number_of_loops**: 
        It represents the number of loops (either for or while) inside each of the functions.
    * **nested_loop_level**: For the functions which have loop inside them, we check if those loop are nested or not; If a nested loop was found, we increse the nested_loop_level by 1. For example, if a function has the nesting level of 2, it means that this function either has a nested loop with depth of 2, or two nested loops with depth of 1. (total number of for blocks + number of first level nested loops)
    * **loop_input_dependent_level**: For this feature, we first creat a list of keywords which includes name of other functions (neighbor functions) and\or name of dependent variables (variables holding the return value of a function). Then for the functions which has loop(s), we check if any of these keywords has been used in the loop's declaration part or not. Each keyword match will cause an increment of 1 for loop_input_dependent_level.
    
  Next table shows the dataset so far:

In [59]:
df

number_of_loops  nested_loop_level  \
ID                                                                            
c233baece621f625b9391c42223520ab969bbf5f               0                0.0   
ebf2053c869201477898f3af1c54f857cc2ec3fa               0                0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c               0                0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c               0                0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c               0                0.0   
...                                                  ...                ...   
e1d0c6c1c29e6ad5164072a5b21340dca7fcb052               0                0.0   
ad677f9687c2e0960d97036f9eb4f2eeda5f690d               0                0.0   
5ce317921ba677aecb71728071b01b2266fd1972               0                0.0   
5ce317921ba677aecb71728071b01b2266fd1972               0                0.0   
c2e2d6621334dc890bbd8a69430012c45a83bf65               0                0.0   

                                          loop_input_dependent_level  
ID                                                                    
c233baece621f625b9391c42223520ab969bbf5f                         0.0  
ebf2053c869201477898f3af1c54f857cc2ec3fa                         0.0  
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c                         0.0  
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c                         0.0  
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c                         0.0  
...                                                              ...  
e1d0c6c1c29e6ad5164072a5b21340dca7fcb052                         0.0  
ad677f9687c2e0960d97036f9eb4f2eeda5f690d                         0.0  
5ce317921ba677aecb71728071b01b2266fd1972                         0.0  
5ce317921ba677aecb71728071b01b2266fd1972                         0.0  
c2e2d6621334dc890bbd8a69430012c45a83bf65                         0.0  

[25653 rows x 3 columns]

### Test:

In [25]:
func_nodes_of_files_tmp = []
with open('/home/amirmahdi/projects/Adaptive-Logging-system-git/code/projects-to-investigate/blobstorageio.py', "r") as file:
        source_code = file.read()
        """here = here+1
        print(here)"""
try:
    functions = extract_functions(source_code)
    for function in functions:
        functions_copy = functions.copy() # Make a copy of list of functions found in current file
        functions_copy.remove(function)    #creating a list of neighbours of current function (in itteratin) by removing itself from functions_copy
        hash_dict[ hashlib.sha1(str.encode(function.name)).hexdigest() ] = [file_dir.split("/")[-1], file_dir, 
                                                                           function.name, functions_copy] 
    func_nodes_of_files_tmp.extend(functions)
except SyntaxError:
    print(SyntaxError)
    pass

In [48]:
x = ""
with open('/home/amirmahdi/projects/Adaptive-Logging-system-git/code/projects-to-investigate/blobstorageio.py', "r") as file: # hash_dict[ID][1] gives the directory of current function's file
    source_code = file.read()
for function_node in func_nodes_of_files_tmp:
    """print('.............................')
    print(function_node.name)"""
    input_dependent_level = 0
    found_loops = loop_finder(function_node) # here we first find loops.
    #ID = hashlib.sha1(str.encode(function_node.name + function_node[1])).hexdigest()
    #file_name = hash_dict[ID][1].split('/')[-1]
    if len(found_loops) != 0: #if any loop was found
        # use "has_dict" to get the neighbour functions of the curent function by using its ID (key):
        #neighbour_functions_names = [n_function.name for n_function in hash_dict[ID][3]]
        
        dependent_variable_names = dependent_variable_finder(source_code)
        
        keywords = dependent_variable_names #+ neighbour_functions_names
        for loop in found_loops: # "loop" here is a node of a loop.
            #print(astor.to_source(loop))
            for keyword in keywords:
                if keyword in astor.to_source(loop).split("\n")[0]:
                    print(function_node.name , keyword)
                    print(astor.to_source(loop).split("\n")[0])
                    input_dependent_level += 1 

delete_files container
for container, blob in zip(containers, blobs):
delete_files  blob 
for container, blob in zip(containers, blobs):
delete_files container
for container, blob in zip(containers, blobs):
delete_files  blob 
for container, blob in zip(containers, blobs):
delete_files containers
for container, blob in zip(containers, blobs):
delete_files containers
for container, blob in zip(containers, blobs):
delete_files  container
for container, blob in zip(containers, blobs):
delete_files  blob 
for container, blob in zip(containers, blobs):
delete_files container
for container, blobs in grouped_blobs.items():
delete_files container
for container, blobs in grouped_blobs.items():
delete_files  blobs 
for container, blobs in grouped_blobs.items():
delete_files  blobs 
for container, blobs in grouped_blobs.items():
delete_files  container
for container, blobs in grouped_blobs.items():
_delete_batch  blob 
for blob in blobs:
_delete_batch  blob 
for blob in blobs:
_delete_batch  blob

In [36]:
for function_node in func_nodes_of_files_tmp:
    print('.............................')
    print(function_node.name)
    input_dependent_level = 0
    nested_loop_level = 0
    found_loops = loop_finder(function_node) # here we first find loops.
    if len(found_loops) != 0: #if any loop was found
        # use "has_dict" to get the neighbour functions of the curent function by using its ID (key):
        neighbour_functions_names = [n_function.name for n_function in hash_dict[ID][3]]
        
        with open(hash_dict[ID][1], "r") as file: # hash_dict[ID][1] gives the directory of current function's file
            source_code = file.read()
        dependent_variable_names = dependent_variable_finder(source_code)
        
        keywords = dependent_variable_names + neighbour_functions_names
        for loop in found_loops: # "loop" here is a node of a loop.
            print(astor.to_source(loop))
            for keyword in keywords:
                if keyword in astor.to_source(loop).split("\n")[0]:
                    print('here2')
                    input_dependent_level += 1 

.............................
parse_azfs_path
.............................
get_azfs_url
.............................
__init__
.............................
__init__
.............................
__init__
.............................
open
.............................
copy
.............................
copy_tree
for entry in self.list_prefix(src):
    rel_path = entry[len(src):]
    try:
        self.copy(entry, dest + rel_path)
        results.append((entry, dest + rel_path, None))
    except BlobStorageError as e:
        results.append((entry, dest + rel_path, e))

.............................
copy_paths
for src_path, dest_path in src_dest_pairs:
    if src_path.endswith('/') and dest_path.endswith('/'):
        try:
            results += self.copy_tree(src_path, dest_path)
        except BlobStorageError as e:
            results.append((src_path, dest_path, e))
    elif not src_path.endswith('/') and not dest_path.endswith('/'):
        try:
            self.copy(src_path, des

In [21]:
with open('/home/amirmahdi/projects/Adaptive-Logging-system-git/code/projects-to-investigate/blobstorageio.py', "r") as file: # hash_dict[ID][1] gives the directory of current function's file
    source_code = file.read()
dependent_variable_names = dependent_variable_finder(source_code)
dependent_variable_names

['_LOGGER ',
 'match ',
 'result ',
 'result ',
 'result ',
 'result ',
 'result ',
 'src_storage_account',
 ' src_container',
 ' src_blob ',
 'dest_container',
 ' dest_blob ',
 'source_blob ',
 'copied_blob ',
 'copy_results ',
 'delete_results ',
 'properties ',
 'container',
 ' blob ',
 'blob_to_check ',
 'container',
 ' blob ',
 'blob_to_delete ',
 'blobs_results ',
 'blobs_results ',
 'paths_to_delete ',
 'containers',
 ' blobs ',
 'containers',
 ' blobs ',
 'final_results ',
 'container_client ',
 'storage_account',
 ' container',
 ' blob ',
 'file_info ',
 'start_time ',
 'container_client ',
 'response ',
 'response ',
 'response ',
 'self._container',
 ' self._blob ',
 'self._blob_to_download ',
 'blob_data ',
 'self._container',
 ' self._blob ',
 'self._content_settings ',
 'self._blob_to_upload ',
 'self._temporary_file ',
 'azure_options ',
 'connect_str ',
 'downloader ',
 'rel_path ',
 "file_status['last_updated'] ",
 'properties ',
 'directory_result ',
 'directory_resul

In [52]:
df.iloc[390]

number_of_loops                  1
nested_loop_level              1.0
loop_input_dependent_level    66.0
Name: 258470721c641df06d29842708ebcba5e4064895, dtype: object

In [53]:
hash_dict["258470721c641df06d29842708ebcba5e4064895"]

['textio_test.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/code/projects-to-investigate/textio_test.py',
 'test_write_pipeline',

In [39]:
count = 0
for i in df["number_of_loops"]:
    count += 1
    print(f"{count}: ", i)

1:  0
2:  0
3:  0
4:  0
5:  0
6:  0
7:  0
8:  1
9:  1
10:  0
11:  2
12:  0
13:  0
14:  0
15:  0
16:  0
17:  0
18:  0
19:  0
20:  2
21:  0
22:  3
23:  1
24:  1
25:  1
26:  0
27:  0
28:  0
29:  0
30:  0
31:  0
32:  0
33:  0
34:  0
35:  0
36:  0
37:  0
38:  0
39:  0
40:  0
41:  0
42:  1
43:  0
44:  0
45:  0
46:  1
47:  0
48:  0
49:  0
50:  0
51:  0
52:  0
53:  0
54:  0
55:  0
56:  0
57:  2
58:  0
59:  0
60:  0
61:  0
62:  0
63:  0
64:  0
65:  0
66:  0
67:  0
68:  1
69:  0
70:  1
71:  0
72:  0
73:  0
74:  0
75:  0
76:  0
77:  0
78:  0
79:  0
80:  0
81:  2
82:  1
83:  1
84:  1
85:  1
86:  0
87:  1
88:  1
89:  0
90:  0
91:  0
92:  0
93:  0
94:  0
95:  0
96:  0
97:  0
98:  0
99:  0
100:  0
101:  0
102:  0
103:  0
104:  0
105:  0
106:  0
107:  0
108:  0
109:  0
110:  0
111:  0
112:  0
113:  0
114:  0
115:  0
116:  0
117:  0
118:  0
119:  1
120:  0
121:  1
122:  0
123:  1
124:  1
125:  0
126:  0
127:  0
128:  0
129:  0
130:  0
131:  0
132:  0
133:  0
134:  0
135:  0
136:  0
137:  0
138:  0
139:

1037:  0
1038:  0
1039:  0
1040:  0
1041:  0
1042:  0
1043:  0
1044:  0
1045:  0
1046:  0
1047:  0
1048:  1
1049:  0
1050:  0
1051:  0
1052:  0
1053:  0
1054:  0
1055:  0
1056:  0
1057:  0
1058:  0
1059:  1
1060:  0
1061:  0
1062:  0
1063:  1
1064:  1
1065:  0
1066:  2
1067:  0
1068:  0
1069:  1
1070:  0
1071:  1
1072:  0
1073:  0
1074:  5
1075:  0
1076:  1
1077:  0
1078:  1
1079:  0
1080:  0
1081:  0
1082:  0
1083:  0
1084:  0
1085:  0
1086:  0
1087:  0
1088:  0
1089:  0
1090:  0
1091:  0
1092:  1
1093:  0
1094:  0
1095:  0
1096:  0
1097:  0
1098:  0
1099:  0
1100:  0
1101:  0
1102:  0
1103:  0
1104:  1
1105:  1
1106:  1
1107:  0
1108:  0
1109:  0
1110:  0
1111:  1
1112:  0
1113:  0
1114:  0
1115:  1
1116:  0
1117:  0
1118:  0
1119:  0
1120:  0
1121:  0
1122:  0
1123:  3
1124:  3
1125:  0
1126:  0
1127:  0
1128:  0
1129:  0
1130:  0
1131:  0
1132:  0
1133:  0
1134:  0
1135:  0
1136:  0
1137:  0
1138:  0
1139:  0
1140:  0
1141:  0
1142:  0
1143:  0
1144:  0
1145:  0
1146:  0
1147:  0
1

3119:  2
3120:  2
3121:  2
3122:  2
3123:  2
3124:  1
3125:  0
3126:  0
3127:  0
3128:  0
3129:  0
3130:  0
3131:  0
3132:  0
3133:  0
3134:  0
3135:  0
3136:  0
3137:  0
3138:  0
3139:  0
3140:  0
3141:  0
3142:  0
3143:  0
3144:  0
3145:  0
3146:  0
3147:  0
3148:  0
3149:  0
3150:  0
3151:  0
3152:  0
3153:  0
3154:  0
3155:  0
3156:  0
3157:  0
3158:  0
3159:  0
3160:  0
3161:  0
3162:  0
3163:  0
3164:  0
3165:  0
3166:  0
3167:  0
3168:  0
3169:  0
3170:  0
3171:  0
3172:  0
3173:  0
3174:  0
3175:  0
3176:  0
3177:  0
3178:  0
3179:  1
3180:  0
3181:  0
3182:  0
3183:  0
3184:  0
3185:  0
3186:  0
3187:  0
3188:  0
3189:  0
3190:  2
3191:  0
3192:  0
3193:  1
3194:  0
3195:  0
3196:  1
3197:  0
3198:  0
3199:  0
3200:  0
3201:  0
3202:  0
3203:  0
3204:  0
3205:  0
3206:  0
3207:  0
3208:  0
3209:  0
3210:  0
3211:  0
3212:  0
3213:  0
3214:  0
3215:  0
3216:  0
3217:  0
3218:  0
3219:  0
3220:  0
3221:  0
3222:  0
3223:  0
3224:  0
3225:  0
3226:  0
3227:  0
3228:  0
3229:  0
3

5193:  0
5194:  0
5195:  0
5196:  0
5197:  0
5198:  0
5199:  0
5200:  0
5201:  0
5202:  0
5203:  0
5204:  0
5205:  0
5206:  0
5207:  0
5208:  0
5209:  0
5210:  0
5211:  1
5212:  0
5213:  0
5214:  0
5215:  0
5216:  0
5217:  0
5218:  0
5219:  0
5220:  0
5221:  0
5222:  0
5223:  0
5224:  0
5225:  0
5226:  0
5227:  0
5228:  0
5229:  0
5230:  0
5231:  0
5232:  0
5233:  0
5234:  0
5235:  0
5236:  0
5237:  0
5238:  0
5239:  0
5240:  0
5241:  0
5242:  0
5243:  0
5244:  1
5245:  0
5246:  0
5247:  1
5248:  1
5249:  0
5250:  0
5251:  0
5252:  0
5253:  0
5254:  0
5255:  0
5256:  0
5257:  0
5258:  0
5259:  0
5260:  0
5261:  0
5262:  0
5263:  0
5264:  0
5265:  0
5266:  0
5267:  0
5268:  1
5269:  0
5270:  0
5271:  0
5272:  0
5273:  0
5274:  0
5275:  0
5276:  0
5277:  0
5278:  0
5279:  0
5280:  0
5281:  0
5282:  1
5283:  0
5284:  0
5285:  0
5286:  0
5287:  0
5288:  0
5289:  0
5290:  0
5291:  0
5292:  0
5293:  0
5294:  0
5295:  0
5296:  0
5297:  0
5298:  0
5299:  0
5300:  0
5301:  0
5302:  0
5303:  0
5

6406:  0
6407:  0
6408:  0
6409:  1
6410:  1
6411:  0
6412:  0
6413:  0
6414:  0
6415:  0
6416:  1
6417:  0
6418:  0
6419:  0
6420:  0
6421:  1
6422:  0
6423:  0
6424:  0
6425:  0
6426:  0
6427:  0
6428:  0
6429:  0
6430:  0
6431:  0
6432:  0
6433:  0
6434:  0
6435:  0
6436:  2
6437:  2
6438:  2
6439:  2
6440:  2
6441:  0
6442:  0
6443:  0
6444:  0
6445:  1
6446:  0
6447:  0
6448:  0
6449:  0
6450:  0
6451:  0
6452:  0
6453:  0
6454:  0
6455:  0
6456:  0
6457:  1
6458:  0
6459:  0
6460:  0
6461:  0
6462:  0
6463:  0
6464:  0
6465:  0
6466:  0
6467:  0
6468:  0
6469:  0
6470:  0
6471:  0
6472:  0
6473:  0
6474:  0
6475:  0
6476:  0
6477:  0
6478:  1
6479:  0
6480:  0
6481:  0
6482:  0
6483:  0
6484:  0
6485:  0
6486:  0
6487:  1
6488:  1
6489:  1
6490:  0
6491:  0
6492:  0
6493:  0
6494:  0
6495:  0
6496:  0
6497:  0
6498:  0
6499:  0
6500:  0
6501:  1
6502:  0
6503:  0
6504:  0
6505:  0
6506:  0
6507:  0
6508:  0
6509:  0
6510:  0
6511:  0
6512:  0
6513:  0
6514:  0
6515:  0
6516:  0
6

...

19286:  0
19287:  0
19288:  0
19289:  0
19290:  0
19291:  0
19292:  0
19293:  0
19294:  0
19295:  0
19296:  0
19297:  0
19298:  0
19299:  0
19300:  0
19301:  0
19302:  0
19303:  0
19304:  0
19305:  0
19306:  0
19307:  0
19308:  0
19309:  0
19310:  0
19311:  0
19312:  0
19313:  0
19314:  0
19315:  0
19316:  0
19317:  1
19318:  0
19319:  0
19320:  0
19321:  1
19322:  1
19323:  0
19324:  0
19325:  0
19326:  0
19327:  0
19328:  0
19329:  0
19330:  0
19331:  0
19332:  0
19333:  0
19334:  0
19335:  0
19336:  14
19337:  0
19338:  14
19339:  0
19340:  0
19341:  0
19342:  0
19343:  0
19344:  0
19345:  0
19346:  0
19347:  0
19348:  0
19349:  0
19350:  0
19351:  0
19352:  0
19353:  0
19354:  0
19355:  0
19356:  0
19357:  1
19358:  1
19359:  1
19360:  1
19361:  2
19362:  2
19363:  1
19364:  0
19365:  0
19366:  0
19367:  0
19368:  0
19369:  0
19370:  1
19371:  0
19372:  0
19373:  0
19374:  0
19375:  0
19376:  0
19377:  0
19378:  0
19379:  0
19380:  0
19381:  0
19382:  0
19383:  0
19384:  0
19385:  

20439:  0
20440:  0
20441:  0
20442:  0
20443:  0
20444:  0
20445:  0
20446:  0
20447:  0
20448:  0
20449:  1
20450:  0
20451:  0
20452:  0
20453:  0
20454:  0
20455:  0
20456:  0
20457:  0
20458:  0
20459:  0
20460:  0
20461:  0
20462:  0
20463:  0
20464:  0
20465:  0
20466:  0
20467:  0
20468:  0
20469:  0
20470:  0
20471:  0
20472:  0
20473:  0
20474:  0
20475:  0
20476:  0
20477:  0
20478:  0
20479:  0
20480:  0
20481:  0
20482:  0
20483:  0
20484:  0
20485:  0
20486:  0
20487:  0
20488:  0
20489:  0
20490:  0
20491:  0
20492:  0
20493:  0
20494:  0
20495:  0
20496:  0
20497:  0
20498:  0
20499:  0
20500:  0
20501:  0
20502:  0
20503:  0
20504:  0
20505:  0
20506:  0
20507:  0
20508:  0
20509:  0
20510:  0
20511:  0
20512:  0
20513:  0
20514:  0
20515:  0
20516:  0
20517:  0
20518:  0
20519:  0
20520:  0
20521:  0
20522:  0
20523:  0
20524:  0
20525:  0
20526:  0
20527:  0
20528:  0
20529:  0
20530:  0
20531:  0
20532:  0
20533:  0
20534:  0
20535:  0
20536:  0
20537:  0
20538:  0


21627:  0
21628:  0
21629:  1
21630:  0
21631:  0
21632:  0
21633:  0
21634:  0
21635:  0
21636:  0
21637:  0
21638:  0
21639:  0
21640:  0
21641:  0
21642:  0
21643:  0
21644:  0
21645:  0
21646:  0
21647:  0
21648:  0
21649:  0
21650:  0
21651:  0
21652:  0
21653:  0
21654:  0
21655:  0
21656:  0
21657:  0
21658:  0
21659:  0
21660:  0
21661:  0
21662:  0
21663:  0
21664:  0
21665:  0
21666:  0
21667:  0
21668:  0
21669:  0
21670:  0
21671:  0
21672:  0
21673:  0
21674:  0
21675:  0
21676:  0
21677:  0
21678:  0
21679:  0
21680:  0
21681:  0
21682:  0
21683:  0
21684:  0
21685:  0
21686:  0
21687:  0
21688:  0
21689:  0
21690:  0
21691:  0
21692:  0
21693:  0
21694:  0
21695:  0
21696:  0
21697:  0
21698:  0
21699:  0
21700:  0
21701:  0
21702:  0
21703:  0
21704:  0
21705:  0
21706:  0
21707:  0
21708:  0
21709:  0
21710:  0
21711:  0
21712:  0
21713:  0
21714:  0
21715:  0
21716:  0
21717:  0
21718:  1
21719:  0
21720:  0
21721:  0
21722:  0
21723:  0
21724:  0
21725:  0
21726:  0


23581:  0
23582:  0
23583:  0
23584:  2
23585:  0
23586:  0
23587:  0
23588:  0
23589:  2
23590:  0
23591:  0
23592:  0
23593:  0
23594:  0
23595:  0
23596:  0
23597:  0
23598:  0
23599:  0
23600:  0
23601:  0
23602:  2
23603:  0
23604:  0
23605:  0
23606:  0
23607:  0
23608:  0
23609:  0
23610:  0
23611:  0
23612:  0
23613:  0
23614:  0
23615:  0
23616:  0
23617:  0
23618:  0
23619:  0
23620:  0
23621:  0
23622:  0
23623:  0
23624:  0
23625:  0
23626:  0
23627:  0
23628:  0
23629:  0
23630:  0
23631:  0
23632:  1
23633:  0
23634:  0
23635:  0
23636:  0
23637:  0
23638:  0
23639:  0
23640:  0
23641:  0
23642:  0
23643:  0
23644:  0
23645:  0
23646:  0
23647:  0
23648:  0
23649:  0
23650:  0
23651:  0
23652:  0
23653:  0
23654:  0
23655:  0
23656:  0
23657:  3
23658:  0
23659:  1
23660:  0
23661:  0
23662:  0
23663:  0
23664:  0
23665:  0
23666:  0
23667:  0
23668:  0
23669:  0
23670:  0
23671:  1
23672:  0
23673:  0
23674:  1
23675:  0
23676:  0
23677:  0
23678:  0
23679:  0
23680:  0


24612:  0
24613:  0
24614:  0
24615:  0
24616:  0
24617:  0
24618:  0
24619:  0
24620:  0
24621:  0
24622:  0
24623:  0
24624:  0
24625:  0
24626:  0
24627:  0
24628:  0
24629:  0
24630:  0
24631:  0
24632:  0
24633:  0
24634:  0
24635:  0
24636:  0
24637:  0
24638:  0
24639:  0
24640:  0
24641:  0
24642:  0
24643:  0
24644:  0
24645:  0
24646:  0
24647:  0
24648:  0
24649:  0
24650:  0
24651:  0
24652:  0
24653:  0
24654:  0
24655:  0
24656:  0
24657:  0
24658:  0
24659:  0
24660:  0
24661:  1
24662:  0
24663:  0
24664:  0
24665:  0
24666:  0
24667:  0
24668:  0
24669:  0
24670:  0
24671:  0
24672:  0
24673:  0
24674:  1
24675:  2
24676:  4
24677:  0
24678:  0
24679:  2
24680:  0
24681:  0
24682:  0
24683:  0
24684:  0
24685:  0
24686:  0
24687:  0
24688:  0
24689:  0
24690:  0
24691:  0
24692:  0
24693:  0
24694:  0
24695:  0
24696:  0
24697:  0
24698:  0
24699:  0
24700:  0
24701:  0
24702:  0
24703:  1
24704:  0
24705:  0
24706:  0
24707:  0
24708:  0
24709:  0
24710:  1
24711:  0


In [51]:
count = 0
for i in df["loop_input_dependent_level"]:
    count += 1
    print(f"{count}: ", i)

1:  0.0
2:  0.0
3:  0.0
4:  0.0
5:  0.0
6:  0.0
7:  0.0
8:  1.0
9:  0.0
10:  0.0
11:  0.0
12:  0.0
13:  0.0
14:  0.0
15:  0.0
16:  0.0
17:  0.0
18:  0.0
19:  0.0
20:  0.0
21:  0.0
22:  8.0
23:  3.0
24:  1.0
25:  0.0
26:  0.0
27:  0.0
28:  0.0
29:  0.0
30:  0.0
31:  0.0
32:  0.0
33:  0.0
34:  0.0
35:  0.0
36:  0.0
37:  0.0
38:  0.0
39:  0.0
40:  0.0
41:  0.0
42:  0.0
43:  0.0
44:  0.0
45:  0.0
46:  0.0
47:  0.0
48:  0.0
49:  0.0
50:  0.0
51:  0.0
52:  0.0
53:  0.0
54:  0.0
55:  0.0
56:  0.0
57:  0.0
58:  0.0
59:  0.0
60:  0.0
61:  0.0
62:  0.0
63:  0.0
64:  0.0
65:  0.0
66:  0.0
67:  0.0
68:  0.0
69:  0.0
70:  0.0
71:  0.0
72:  0.0
73:  0.0
74:  0.0
75:  0.0
76:  0.0
77:  0.0
78:  0.0
79:  0.0
80:  0.0
81:  0.0
82:  0.0
83:  0.0
84:  0.0
85:  0.0
86:  0.0
87:  0.0
88:  0.0
89:  0.0
90:  0.0
91:  0.0
92:  0.0
93:  0.0
94:  0.0
95:  0.0
96:  0.0
97:  0.0
98:  0.0
99:  0.0
100:  0.0
101:  0.0
102:  0.0
103:  0.0
104:  0.0
105:  0.0
106:  0.0
107:  0.0
108:  0.0
109:  0.0
110:  0.0
111:  0.

1009:  0.0
1010:  0.0
1011:  0.0
1012:  0.0
1013:  0.0
1014:  0.0
1015:  0.0
1016:  0.0
1017:  0.0
1018:  0.0
1019:  0.0
1020:  0.0
1021:  0.0
1022:  0.0
1023:  0.0
1024:  0.0
1025:  0.0
1026:  0.0
1027:  0.0
1028:  1.0
1029:  0.0
1030:  0.0
1031:  0.0
1032:  0.0
1033:  0.0
1034:  0.0
1035:  0.0
1036:  0.0
1037:  0.0
1038:  0.0
1039:  0.0
1040:  0.0
1041:  0.0
1042:  0.0
1043:  0.0
1044:  0.0
1045:  0.0
1046:  0.0
1047:  0.0
1048:  0.0
1049:  0.0
1050:  0.0
1051:  0.0
1052:  0.0
1053:  0.0
1054:  0.0
1055:  0.0
1056:  0.0
1057:  0.0
1058:  0.0
1059:  0.0
1060:  0.0
1061:  0.0
1062:  0.0
1063:  0.0
1064:  0.0
1065:  0.0
1066:  0.0
1067:  0.0
1068:  0.0
1069:  0.0
1070:  0.0
1071:  0.0
1072:  0.0
1073:  0.0
1074:  0.0
1075:  0.0
1076:  0.0
1077:  0.0
1078:  0.0
1079:  0.0
1080:  0.0
1081:  0.0
1082:  0.0
1083:  0.0
1084:  0.0
1085:  0.0
1086:  0.0
1087:  0.0
1088:  0.0
1089:  0.0
1090:  0.0
1091:  0.0
1092:  0.0
1093:  0.0
1094:  0.0
1095:  0.0
1096:  0.0
1097:  0.0
1098:  0.0
1099:  0.0

2315:  0.0
2316:  0.0
2317:  0.0
2318:  0.0
2319:  0.0
2320:  0.0
2321:  0.0
2322:  0.0
2323:  0.0
2324:  0.0
2325:  0.0
2326:  0.0
2327:  0.0
2328:  0.0
2329:  0.0
2330:  0.0
2331:  0.0
2332:  0.0
2333:  0.0
2334:  0.0
2335:  0.0
2336:  0.0
2337:  0.0
2338:  0.0
2339:  0.0
2340:  0.0
2341:  0.0
2342:  0.0
2343:  0.0
2344:  0.0
2345:  0.0
2346:  0.0
2347:  0.0
2348:  0.0
2349:  0.0
2350:  0.0
2351:  0.0
2352:  0.0
2353:  0.0
2354:  0.0
2355:  0.0
2356:  0.0
2357:  0.0
2358:  0.0
2359:  0.0
2360:  0.0
2361:  1.0
2362:  0.0
2363:  0.0
2364:  0.0
2365:  0.0
2366:  0.0
2367:  0.0
2368:  0.0
2369:  0.0
2370:  0.0
2371:  0.0
2372:  0.0
2373:  0.0
2374:  0.0
2375:  0.0
2376:  0.0
2377:  0.0
2378:  0.0
2379:  0.0
2380:  0.0
2381:  0.0
2382:  0.0
2383:  0.0
2384:  0.0
2385:  0.0
2386:  0.0
2387:  0.0
2388:  0.0
2389:  0.0
2390:  0.0
2391:  0.0
2392:  0.0
2393:  0.0
2394:  0.0
2395:  0.0
2396:  0.0
2397:  0.0
2398:  0.0
2399:  0.0
2400:  0.0
2401:  0.0
2402:  0.0
2403:  0.0
2404:  0.0
2405:  0.0

3412:  0.0
3413:  1.0
3414:  0.0
3415:  0.0
3416:  0.0
3417:  0.0
3418:  0.0
3419:  0.0
3420:  0.0
3421:  0.0
3422:  0.0
3423:  0.0
3424:  0.0
3425:  0.0
3426:  0.0
3427:  0.0
3428:  0.0
3429:  0.0
3430:  0.0
3431:  0.0
3432:  0.0
3433:  0.0
3434:  0.0
3435:  0.0
3436:  0.0
3437:  0.0
3438:  0.0
3439:  0.0
3440:  0.0
3441:  0.0
3442:  0.0
3443:  0.0
3444:  0.0
3445:  0.0
3446:  2.0
3447:  0.0
3448:  0.0
3449:  0.0
3450:  0.0
3451:  0.0
3452:  0.0
3453:  0.0
3454:  0.0
3455:  0.0
3456:  0.0
3457:  0.0
3458:  0.0
3459:  0.0
3460:  0.0
3461:  0.0
3462:  0.0
3463:  0.0
3464:  2.0
3465:  0.0
3466:  0.0
3467:  1.0
3468:  0.0
3469:  0.0
3470:  0.0
3471:  0.0
3472:  0.0
3473:  0.0
3474:  1.0
3475:  0.0
3476:  0.0
3477:  0.0
3478:  0.0
3479:  0.0
3480:  0.0
3481:  0.0
3482:  0.0
3483:  0.0
3484:  1.0
3485:  0.0
3486:  0.0
3487:  0.0
3488:  1.0
3489:  0.0
3490:  0.0
3491:  0.0
3492:  0.0
3493:  0.0
3494:  0.0
3495:  0.0
3496:  0.0
3497:  0.0
3498:  0.0
3499:  0.0
3500:  0.0
3501:  0.0
3502:  0.0

4376:  0.0
4377:  0.0
4378:  0.0
4379:  0.0
4380:  0.0
4381:  0.0
4382:  0.0
4383:  0.0
4384:  0.0
4385:  0.0
4386:  0.0
4387:  0.0
4388:  0.0
4389:  0.0
4390:  0.0
4391:  0.0
4392:  0.0
4393:  0.0
4394:  0.0
4395:  0.0
4396:  0.0
4397:  0.0
4398:  0.0
4399:  0.0
4400:  0.0
4401:  0.0
4402:  0.0
4403:  0.0
4404:  0.0
4405:  0.0
4406:  0.0
4407:  0.0
4408:  0.0
4409:  0.0
4410:  0.0
4411:  0.0
4412:  0.0
4413:  0.0
4414:  0.0
4415:  0.0
4416:  0.0
4417:  0.0
4418:  0.0
4419:  0.0
4420:  0.0
4421:  0.0
4422:  0.0
4423:  0.0
4424:  0.0
4425:  0.0
4426:  0.0
4427:  0.0
4428:  0.0
4429:  0.0
4430:  0.0
4431:  0.0
4432:  0.0
4433:  0.0
4434:  0.0
4435:  0.0
4436:  0.0
4437:  0.0
4438:  0.0
4439:  0.0
4440:  0.0
4441:  0.0
4442:  0.0
4443:  0.0
4444:  0.0
4445:  0.0
4446:  0.0
4447:  0.0
4448:  0.0
4449:  0.0
4450:  0.0
4451:  0.0
4452:  0.0
4453:  0.0
4454:  0.0
4455:  0.0
4456:  0.0
4457:  0.0
4458:  0.0
4459:  0.0
4460:  0.0
4461:  0.0
4462:  0.0
4463:  0.0
4464:  0.0
4465:  0.0
4466:  0.0

...

21546:  0.0
21547:  0.0
21548:  0.0
21549:  0.0
21550:  0.0
21551:  0.0
21552:  0.0
21553:  0.0
21554:  0.0
21555:  0.0
21556:  0.0
21557:  0.0
21558:  0.0
21559:  0.0
21560:  0.0
21561:  0.0
21562:  0.0
21563:  0.0
21564:  0.0
21565:  0.0
21566:  0.0
21567:  0.0
21568:  0.0
21569:  0.0
21570:  0.0
21571:  0.0
21572:  0.0
21573:  0.0
21574:  0.0
21575:  0.0
21576:  0.0
21577:  0.0
21578:  0.0
21579:  0.0
21580:  0.0
21581:  0.0
21582:  0.0
21583:  0.0
21584:  0.0
21585:  0.0
21586:  0.0
21587:  0.0
21588:  0.0
21589:  0.0
21590:  0.0
21591:  0.0
21592:  0.0
21593:  0.0
21594:  0.0
21595:  0.0
21596:  0.0
21597:  0.0
21598:  0.0
21599:  0.0
21600:  0.0
21601:  0.0
21602:  0.0
21603:  0.0
21604:  0.0
21605:  0.0
21606:  0.0
21607:  0.0
21608:  0.0
21609:  0.0
21610:  0.0
21611:  0.0
21612:  0.0
21613:  0.0
21614:  0.0
21615:  0.0
21616:  0.0
21617:  0.0
21618:  0.0
21619:  0.0
21620:  0.0
21621:  0.0
21622:  0.0
21623:  0.0
21624:  0.0
21625:  0.0
21626:  0.0
21627:  0.0
21628:  0.0
2162

22528:  0.0
22529:  0.0
22530:  0.0
22531:  0.0
22532:  0.0
22533:  0.0
22534:  0.0
22535:  0.0
22536:  0.0
22537:  0.0
22538:  0.0
22539:  0.0
22540:  0.0
22541:  0.0
22542:  0.0
22543:  0.0
22544:  0.0
22545:  0.0
22546:  0.0
22547:  0.0
22548:  0.0
22549:  0.0
22550:  0.0
22551:  0.0
22552:  0.0
22553:  0.0
22554:  0.0
22555:  0.0
22556:  0.0
22557:  0.0
22558:  1.0
22559:  0.0
22560:  0.0
22561:  0.0
22562:  0.0
22563:  0.0
22564:  0.0
22565:  0.0
22566:  0.0
22567:  0.0
22568:  0.0
22569:  0.0
22570:  0.0
22571:  0.0
22572:  0.0
22573:  0.0
22574:  0.0
22575:  1.0
22576:  0.0
22577:  0.0
22578:  0.0
22579:  0.0
22580:  0.0
22581:  0.0
22582:  0.0
22583:  0.0
22584:  0.0
22585:  0.0
22586:  0.0
22587:  0.0
22588:  0.0
22589:  0.0
22590:  0.0
22591:  0.0
22592:  0.0
22593:  0.0
22594:  0.0
22595:  0.0
22596:  0.0
22597:  0.0
22598:  0.0
22599:  0.0
22600:  0.0
22601:  0.0
22602:  0.0
22603:  0.0
22604:  0.0
22605:  0.0
22606:  0.0
22607:  0.0
22608:  0.0
22609:  0.0
22610:  0.0
2261

23355:  0.0
23356:  0.0
23357:  0.0
23358:  0.0
23359:  0.0
23360:  0.0
23361:  0.0
23362:  0.0
23363:  0.0
23364:  0.0
23365:  0.0
23366:  0.0
23367:  0.0
23368:  1.0
23369:  0.0
23370:  0.0
23371:  0.0
23372:  0.0
23373:  0.0
23374:  0.0
23375:  0.0
23376:  0.0
23377:  0.0
23378:  0.0
23379:  0.0
23380:  0.0
23381:  0.0
23382:  0.0
23383:  0.0
23384:  1.0
23385:  0.0
23386:  0.0
23387:  0.0
23388:  0.0
23389:  0.0
23390:  0.0
23391:  0.0
23392:  0.0
23393:  0.0
23394:  0.0
23395:  0.0
23396:  0.0
23397:  0.0
23398:  0.0
23399:  0.0
23400:  0.0
23401:  0.0
23402:  0.0
23403:  0.0
23404:  0.0
23405:  0.0
23406:  0.0
23407:  0.0
23408:  0.0
23409:  0.0
23410:  0.0
23411:  0.0
23412:  0.0
23413:  0.0
23414:  0.0
23415:  0.0
23416:  0.0
23417:  0.0
23418:  0.0
23419:  0.0
23420:  0.0
23421:  2.0
23422:  0.0
23423:  0.0
23424:  0.0
23425:  0.0
23426:  0.0
23427:  0.0
23428:  0.0
23429:  0.0
23430:  0.0
23431:  0.0
23432:  0.0
23433:  0.0
23434:  0.0
23435:  0.0
23436:  0.0
23437:  0.0
2343

24508:  0.0
24509:  0.0
24510:  0.0
24511:  0.0
24512:  0.0
24513:  0.0
24514:  0.0
24515:  0.0
24516:  0.0
24517:  0.0
24518:  0.0
24519:  0.0
24520:  0.0
24521:  0.0
24522:  0.0
24523:  0.0
24524:  0.0
24525:  0.0
24526:  0.0
24527:  0.0
24528:  0.0
24529:  0.0
24530:  0.0
24531:  0.0
24532:  0.0
24533:  0.0
24534:  0.0
24535:  0.0
24536:  0.0
24537:  0.0
24538:  0.0
24539:  0.0
24540:  0.0
24541:  0.0
24542:  0.0
24543:  0.0
24544:  0.0
24545:  0.0
24546:  0.0
24547:  0.0
24548:  0.0
24549:  0.0
24550:  0.0
24551:  0.0
24552:  0.0
24553:  0.0
24554:  0.0
24555:  0.0
24556:  0.0
24557:  0.0
24558:  0.0
24559:  0.0
24560:  0.0
24561:  0.0
24562:  0.0
24563:  0.0
24564:  0.0
24565:  0.0
24566:  0.0
24567:  0.0
24568:  0.0
24569:  0.0
24570:  0.0
24571:  0.0
24572:  0.0
24573:  0.0
24574:  0.0
24575:  2.0
24576:  0.0
24577:  0.0
24578:  0.0
24579:  0.0
24580:  0.0
24581:  0.0
24582:  0.0
24583:  0.0
24584:  0.0
24585:  0.0
24586:  0.0
24587:  0.0
24588:  0.0
24589:  0.0
24590:  0.0
2459

25551:  0.0
25552:  4.0
25553:  1.0
25554:  1.0
25555:  0.0
25556:  27.0
25557:  1.0
25558:  2.0
25559:  1.0
25560:  0.0
25561:  0.0
25562:  0.0
25563:  0.0
25564:  0.0
25565:  0.0
25566:  0.0
25567:  0.0
25568:  0.0
25569:  0.0
25570:  0.0
25571:  0.0
25572:  0.0
25573:  0.0
25574:  0.0
25575:  0.0
25576:  0.0
25577:  0.0
25578:  0.0
25579:  0.0
25580:  0.0
25581:  0.0
25582:  0.0
25583:  0.0
25584:  0.0
25585:  0.0
25586:  0.0
25587:  0.0
25588:  0.0
25589:  0.0
25590:  0.0
25591:  0.0
25592:  0.0
25593:  0.0
25594:  0.0
25595:  0.0
25596:  0.0
25597:  0.0
25598:  0.0
25599:  0.0
25600:  0.0
25601:  0.0
25602:  0.0
25603:  0.0
25604:  0.0
25605:  0.0
25606:  0.0
25607:  0.0
25608:  0.0
25609:  0.0
25610:  0.0
25611:  0.0
25612:  0.0
25613:  0.0
25614:  0.0
25615:  0.0
25616:  0.0
25617:  0.0
25618:  0.0
25619:  0.0
25620:  36.0
25621:  0.0
25622:  0.0
25623:  0.0
25624:  0.0
25625:  0.0
25626:  0.0
25627:  0.0
25628:  0.0
25629:  0.0
25630:  0.0
25631:  0.0
25632:  0.0
25633:  0.0
25

## 2. Synchronization

* #### Defining needed function:

In [61]:
"""This function is to return the number of occurence of a specific function call, by taking the function's name
and the AST node (which we want to investigate) as its input."""

def func_call_finder(func_call_to_find, node):
    found_func_call = 0
    #tmp = ast.iter_child_nodes(node)
    tmp = ast.walk(node)
    for child_node in tmp:
        try:
            #print("try")
            if isinstance(child_node, ast.Call):
                #print("isinstance")
                if child_node.func.id == func_call_to_find: # child_node.func.attr: this returns the function call's name
                    found_func_call += 1
                    print(child_node.func.id)
        except:
            #print("attribute type of function call found")
            try:
                if child_node.func.attr == func_call_to_find:
                    if func_call_to_find == "join": # this if is to remove .join() functions which are for string concatination
                        #print(astor.to_source(child_node).split(".join")[-2])
                        if "\"" in astor.to_source(child_node).split(".join")[-2]:
                            print("string")
                        else:
                            found_func_call += 1
                            print("attr:" + child_node.func.attr)
                    else:
                        found_func_call += 1
                        print("attr:" + child_node.func.attr)
            except:
                pass
        
    return found_func_call

* #### Adding 6 more synchronization related features to the Dataframe:

    When it comes to synchronization in python there are some specific funciton calls that are being used, namely:
    
    * **threading.Thread()**: Defining thread and setting target function to be processed under that thread.
    * **Thread.start()**: Letting Threads start.
    * **Thread.join()**: wait until the thread finish their job.
    * **threading.Lock()**: Defining a lock. It is to be passed as argument to the defined thread and also argument of the function under the thread.
    * **Lock.acquire()**
    * **Lock.release()**
    
  In this part, we itereate over functions under investigation and extract the number of occurence of the function calls mentioned above by using "func_call_finder" function. It is worth mentioning that "join" is a function call that also can be used for string concationation, so that "func_call_finder" recognizes and ignores them in counting progress.
  
* #### Limitation:
     * 1. As it can be seen threading process is distributed among different functions, so that to model the exact behaviour of it we need to consider more functions at once, while we are studig only one function at a time.
     * 2. In our fetures we do not check the threading.thread() arguments. The arguments can add more information and details(like if developers have passed lock in arguments, if so is it correct one? etc). However, we are not able to do this in this go, since each project may have different standard in chosing name for the locks they define,,,
     
     
<img src= "images/thread_synchronization.png" alt="alt text" longdesc="https://www.geeksforgeeks.org/multithreading-in-python-set-2-synchronization/" title="From W3Schools"/>

  

In [62]:
for function_node in func_nodes_of_files:
    number_of_defined_threads = func_call_finder("Thread", function_node[0])
    #print(number_of_defined_threads)
    number_of_started_threads = func_call_finder("start", function_node[0])
    #print(number_of_started_threads)
    number_of_join_threads = func_call_finder("join", function_node[0])
    #print(number_of_join_threads)
    number_of_defined_locks = func_call_finder("Lock", function_node[0])
    #print(number_of_defined_locks)
    number_of_acquired_locks_threads = func_call_finder("acquire", function_node[0])
    #print(number_of_acquired_locks_threads)
    number_of_released_locks = func_call_finder("release", function_node[0])
    #print(number_of_released_locks)
    #print("...........................................")
    
    ID = hashlib.sha1(str.encode(function_node[0].name + function_node[1])).hexdigest()
    df.at[ID, "number_of_defined_threads"] = int(number_of_defined_threads)
    df.at[ID, "number_of_started_threads"] = int(number_of_started_threads)
    df.at[ID, "number_of_join_threads"] = int(number_of_join_threads)
    df.at[ID, "number_of_defined_locks"] = int(number_of_defined_locks)
    df.at[ID, "number_of_acquired_locks_threads"] = int(number_of_acquired_locks_threads)
    df.at[ID, "number_of_released_locks"] = int(number_of_released_locks)
    

attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:Thread
attr:Thread
attr:start
attr:start
attr:join
string
string
string
string
attr:join
attr:join
attr:join
attr:join
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:start
string
string
string
string
string
attr:join
attr:join
attr:join
attr:Thread
attr:start
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:start
attr:start
attr:join
attr:Lock
attr:acquire
attr:release
attr:join
attr:acquire
attr:release
attr:acquire
attr:release
attr:Thread
attr:start
attr:join
attr:Lock
attr:start
attr:start
attr:start
attr:start
attr:Thread
attr:start
attr:join
attr:Thread
attr:start
attr:join


attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:join
string
string
string
string
attr:join
string
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
string
string
string
string
attr:acquire
attr:release
attr:release
attr:start
attr:start
attr:start
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:start
attr:release
attr:acquire
attr:join
attr:join
attr:join
attr:Lock
attr:Lock
attr:start
attr:Lock
attr:start
attr:start
attr:start
string
string
Thread
Thread
Thread
Thread
Thread
Thread
Thread
Thread
Thread
Thread
Thread
Thread
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:start
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
Thread
attr:start
attr:join
Lock
string
string
string
string
string
string

string
string
attr:start
attr:start
attr:start
attr:start
string
string
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
attr:join
string
string
string
string
string
string
string
string
string
attr:join
string
string
string
attr:join
string
string
attr:join
string
attr:Lock
string
string
string
string
string
string
attr:join
attr:join
attr:start
string
string
string
attr:Lock
attr:Lock
attr:Lock
attr:join
attr:join
attr:start
attr:start
attr:Lock
string
attr:start
attr:join
attr:join
attr:join
attr:start
attr:start
attr:start
attr:start
attr:join
string
attr:Thread
attr:start
attr:join
attr:Thread
attr:start
attr:join
attr:join
attr:start
attr:start
attr:Lock
attr:acquire
attr:release
attr:acquire
attr:release
attr:acquire
attr:release
attr:start
attr:join
attr:join
attr:start
attr:start
attr:start
attr:start
attr:Thread
attr:start
attr:Lock
attr:join
string
string
attr:Thread
attr:start
attr:join
string
attr:start
attr:start
attr:start
attr:start
string

* #### Final version of the Dataset so far:

In [34]:
df

number_of_loops  nested_loop_level  \
ID                                                                            
ee2c971f20132061d7e749267b166730ff734a7c               0                0.0   
d4f01649de54f7e70c368da6571ac5ccda9ddf79               0                0.0   
bae8ac45f183613712be11aee11ec93b6e36e0d0               0                0.0   
9aa1369d290e7732b7ca8db743f27d10c0ace093               0                0.0   
9aa1369d290e7732b7ca8db743f27d10c0ace093               0                0.0   
...                                                  ...                ...   
4553a4f2a833f635e06b21120fd95dbd7dc3fe09               0                0.0   
05bb76f1e435db71754a65f9e8acec7dfdd4c65d               0                0.0   
4fb170389e1b482ed3c7fae2967caca7629c84a6               0                0.0   
ea00c9c71a474f84e3280537f04ede09ae70e44f               0                0.0   
fd5e6143d36ce5ffa6caff953de41436984b7fc3               0                0.0   

                                          loop_input_dependent_level  \
ID                                                                     
ee2c971f20132061d7e749267b166730ff734a7c                         0.0   
d4f01649de54f7e70c368da6571ac5ccda9ddf79                         0.0   
bae8ac45f183613712be11aee11ec93b6e36e0d0                         0.0   
9aa1369d290e7732b7ca8db743f27d10c0ace093                         0.0   
9aa1369d290e7732b7ca8db743f27d10c0ace093                         0.0   
...                                                              ...   
4553a4f2a833f635e06b21120fd95dbd7dc3fe09                         0.0   
05bb76f1e435db71754a65f9e8acec7dfdd4c65d                         0.0   
4fb170389e1b482ed3c7fae2967caca7629c84a6                         0.0   
ea00c9c71a474f84e3280537f04ede09ae70e44f                         0.0   
fd5e6143d36ce5ffa6caff953de41436984b7fc3                         0.0   

                                          number_of_defined_threads  \
ID                                                                    
ee2c971f20132061d7e749267b166730ff734a7c                        0.0   
d4f01649de54f7e70c368da6571ac5ccda9ddf79                        0.0   
bae8ac45f183613712be11aee11ec93b6e36e0d0                        0.0   
9aa1369d290e7732b7ca8db743f27d10c0ace093                        0.0   
9aa1369d290e7732b7ca8db743f27d10c0ace093                        0.0   
...                                                             ...   
4553a4f2a833f635e06b21120fd95dbd7dc3fe09                        0.0   
05bb76f1e435db71754a65f9e8acec7dfdd4c65d                        0.0   
4fb170389e1b482ed3c7fae2967caca7629c84a6                        0.0   
ea00c9c71a474f84e3280537f04ede09ae70e44f                        0.0   
fd5e6143d36ce5ffa6caff953de41436984b7fc3                        0.0   

                                          number_of_started_threads  \
ID                                                                    
ee2c971f20132061d7e749267b166730ff734a7c                        0.0   
d4f01649de54f7e70c368da6571ac5ccda9ddf79                        0.0   
bae8ac45f183613712be11aee11ec93b6e36e0d0                        0.0   
9aa1369d290e7732b7ca8db743f27d10c0ace093                        0.0   
9aa1369d290e7732b7ca8db743f27d10c0ace093                        0.0   
...                                                             ...   
4553a4f2a833f635e06b21120fd95dbd7dc3fe09                        0.0   
05bb76f1e435db71754a65f9e8acec7dfdd4c65d                        0.0   
4fb170389e1b482ed3c7fae2967caca7629c84a6                        0.0   
ea00c9c71a474f84e3280537f04ede09ae70e44f                        0.0   
fd5e6143d36ce5ffa6caff953de41436984b7fc3                        0.0   

                                          number_of_join_threads  \
ID                                                                 
ee2c971f20132061d7e749267b166730ff734a7c                     0.0   
d4f01649de54f7e70c368da6

## 3. API Misues

### 3.1. Django ORM API Misuses

There are a few resons for performance regression when it comes to using ORM APIs to query the database; **1)** Making complex query, which makes it harder for ORM to transform it to actual database query. **2)** Retrieving too much data in a query which miht not be neccassary (this can be handle by using selec_related() and prefetch_related() ). **3)** Not using database_level constraints, for instance, sorting the records after fetching data from database instead of sorting the database once. Following APIs are the ones could potentially cause one of the performance regressions mentioned above.

* overusing **extra()** .(using sub-queries which make a query more complex and it is better to join them)
* Overusing **order_by()** .(not using database_level constraints)
* Overusing **select_related()**. (on the other hand, not using it might lead to retrieving too much data)
* Using too many **OR** conditions in queries. (make queries complex)
* Not using **prefetch_related()** .(may cause retrieving too much data)
* Not using **transactoins**
* Not using **Bulk operations**
* Not using **cashing**
* Using **values()** or **values_list()** without a specific set of fields

These are factors that would have an impact on performance, which are mentioned in Django documantation. 

*References*: [ORM paper](https://dl.acm.org/doi/abs/10.1145/3180155.3180194?casa_token=DDifZ_1ncBQAAAAA:awTyreq54BC65QWFebCMs8Hkxdc0w8I0guhdUwKCEAUmpD_GgQHA6jrviHXk6xHaBxMOdipjpIjvzQ) , Django documentation and chat GPT.

In [63]:
for function_node in func_nodes_of_files:
    number_of_usage_of_extra = func_call_finder("extra", function_node[0])
    #print(number_of_defined_threads)
    number_of_usage_of_order_by = func_call_finder("order_by", function_node[0])
    #print(number_of_started_threads)
    number_of_usage_of_select_related = func_call_finder("select_related", function_node[0])
    #print(number_of_acquired_locks_threads)
    #print("...........................................")
    
    ID = hashlib.sha1(str.encode(function_node[0].name + function_node[1])).hexdigest()
    df.at[ID, "number_of_usage_of_extra"] = int(number_of_usage_of_extra)
    #df.at[ID, "number_of_usage_of_order_by"] = int(number_of_usage_of_order_by)
    #df.at[ID, "number_of_usage_of_values"] = int(number_of_usage_of_values)
    df.at[ID, "number_of_usage_of_values_list"] = int(number_of_usage_of_order_by)
    df.at[ID, "number_of_usage_of_select_related"] = int(number_of_usage_of_select_related)

In [64]:
df

number_of_loops  nested_loop_level  \
ID                                                                            
c233baece621f625b9391c42223520ab969bbf5f               0                0.0   
ebf2053c869201477898f3af1c54f857cc2ec3fa               0                0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c               0                0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c               0                0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c               0                0.0   
...                                                  ...                ...   
e1d0c6c1c29e6ad5164072a5b21340dca7fcb052               0                0.0   
ad677f9687c2e0960d97036f9eb4f2eeda5f690d               0                0.0   
5ce317921ba677aecb71728071b01b2266fd1972               0                0.0   
5ce317921ba677aecb71728071b01b2266fd1972               0                0.0   
c2e2d6621334dc890bbd8a69430012c45a83bf65               0                0.0   

                                          loop_input_dependent_level  \
ID                                                                     
c233baece621f625b9391c42223520ab969bbf5f                         0.0   
ebf2053c869201477898f3af1c54f857cc2ec3fa                         0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c                         0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c                         0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c                         0.0   
...                                                              ...   
e1d0c6c1c29e6ad5164072a5b21340dca7fcb052                         0.0   
ad677f9687c2e0960d97036f9eb4f2eeda5f690d                         0.0   
5ce317921ba677aecb71728071b01b2266fd1972                         0.0   
5ce317921ba677aecb71728071b01b2266fd1972                         0.0   
c2e2d6621334dc890bbd8a69430012c45a83bf65                         0.0   

                                          number_of_defined_threads  \
ID                                                                    
c233baece621f625b9391c42223520ab969bbf5f                        0.0   
ebf2053c869201477898f3af1c54f857cc2ec3fa                        0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c                        0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c                        0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c                        0.0   
...                                                             ...   
e1d0c6c1c29e6ad5164072a5b21340dca7fcb052                        0.0   
ad677f9687c2e0960d97036f9eb4f2eeda5f690d                        0.0   
5ce317921ba677aecb71728071b01b2266fd1972                        0.0   
5ce317921ba677aecb71728071b01b2266fd1972                        0.0   
c2e2d6621334dc890bbd8a69430012c45a83bf65                        0.0   

                                          number_of_started_threads  \
ID                                                                    
c233baece621f625b9391c42223520ab969bbf5f                        0.0   
ebf2053c869201477898f3af1c54f857cc2ec3fa                        0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c                        0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c                        0.0   
1939f247b3ae06244a8d50fa7dabbcc975c6fc4c                        0.0   
...                                                             ...   
e1d0c6c1c29e6ad5164072a5b21340dca7fcb052                        0.0   
ad677f9687c2e0960d97036f9eb4f2eeda5f690d                        0.0   
5ce317921ba677aecb71728071b01b2266fd1972                        0.0   
5ce317921ba677aecb71728071b01b2266fd1972                        0.0   
c2e2d6621334dc890bbd8a69430012c45a83bf65                        0.0   

                                          number_of_join_threads  \
ID                                                                 
c233baece621f625b9391c42223520ab969bbf5f                     0.0   
ebf2053c869201477898f3af

In [65]:
df.to_csv("data/dataframe_main.csv")

In [14]:
df = pd.read_csv("data/dataframe.csv").set_index("ID")

### 3.2. DL API Misuses

In [21]:
None

### 3.3. ML Cloud API Misuses

In [22]:
None

# Data Preprocessing 

In [17]:
df.iloc[1].astype(np.float32).sum() >= 0

True

In [18]:
len(df.columns)

12

## 1.Data Normalization

(it is not needed for current implementaion of reward function)

In [ ]:
df_copy = df.copy()
df_normalized = df_copy.apply(lambda iterator : (iterator - iterator.mean()))

# Garbage:

In [ ]:
# this loops iterates over all of the extracted functions to find the input_dependent_level and nested_loop_level
for function_node in func_nodes_of_files:
    input_dependent_level = 0
    nested_loop_level = 0
    found_loops = loop_finder(function_node[0]) # here we first find loops.
    ID = hashlib.sha1(str.encode(function_node[0].name + function_node[1])).hexdigest()
    if len(found_loops) != 0: #if any loop was found
        
        # use "has_dict" to get the neighbour functions of the curent function by using its ID (key):
        neighbour_functions_names = [n_function.name for n_function in hash_dict[ID][3]]
        
        with open(hash_dict[ID][1], "r") as file: # hash_dict[ID][1] gives the directory of current function's file
            source_code = file.read()
        dependent_variable_names = dependent_variable_finder(source_code)
        
        keywords = dependent_variable_names + neighbour_functions_names
        for loop in found_loops: # "loop" here is a node of a loop.
            for keyword in keywords:
                if keyword in astor.to_source(loop).split("\n")[0]:
                    input_dependent_level += 1     
    nested_loop_level = nested_loop_level_finder(found_loops)
    df.at[ID, "nested_loop_level"] = int(nested_loop_level)
    df.at[ID, "loop_input_dependent_level"] = int(input_dependent_level)
# the functions without any loops inside will have Nan value for "loop_input_dependent_level", so we replace them:
df["nested_loop_level"] = df["nested_loop_level"].fillna(0)
df["loop_input_dependent_level"] = df["loop_input_dependent_level"].fillna(0) 

In [12]:
"""This part is to test the nested_loop_level_finder function we defined above"""

s = "for loop in found_loops:\n for keyword in keywords:\n      if keyword in astor.to_source(loop):\n       input_dependent_level += 1"
tree = ast.parse(s)
nested_loop_level_finder(tree)

1

In [ ]:

def nested_loop_level_finder(loop_node):
    #nodes_list = ast.walk(loop_node)
    nodes_list = ast.iter_child_nodes(loop_node) # this returns a generator
    for child_node in nodes_list: # iterate over nodes_list which is a generator.
        print('child node: ')
        print(astor.to_source(child_node))
        if isinstance(child_node, ast.For) or isinstance(child_node, ast.While):
            return 1 + nested_loop_level_finder(child_node)
        elif child_node is not next(nodes_list, None): #checks if it is last item of the generator 
            return 0

In [45]:
fors = [] #this should be dataframe instead
for function_node in func_nodes_of_files:
    found_fors = loop_finder(function_node)
    if len(found_fors) != 0:
        fors.extend(found_fors)
print(astor.to_source(for_nodes_of_files[0][0]))

for node in ([obj, target] + (related_nodes or [])):
    if isinstance(node, Project):
        create_timelines.post(node.node_id)



In [13]:
functions = []
for file_dir in file_directories:
    with open(file_dir, "r") as file:
        source_code = file.read()
    functions.extend(dependent_variable_finder(source_code))
print(len(functions))
#print(astor.to_source(functions[10]))
print(functions[10].func.id)

2240


AttributeError: 'str' object has no attribute 'func'

In [58]:
"""codes from ast documentation to check function calls and ways to access to their names and atributes"""

print(ast.dump(ast.parse('"_"', mode='eval')))
print(isinstance(ast.dump(ast.parse('"_".func(a, b=c, *d, **e)', mode='eval')), ast.Call))
print(isinstance(ast.dump(ast.parse('"_".func(a, b=c, *d, **e)', mode='eval')), ast.Call))
print(ast.parse('t.func(a, b=c, *d, **e)', mode='eval').body.func.value.id)
print(ast.parse('t.p.func(a, b=c, *d, **e)', mode='eval').body.func.attr)
print(ast.dump(ast.parse('t.p.func(a, b=c, *d, **e)', mode='eval'))) 
print(ast.parse('t.p.func(a, b=c, *d, **e)', mode='eval').body.func.value.value.id) # important
print(ast.parse('t.p.func(a, b=c, *d, **e)', mode='eval').body.func.value.attr) # importatn
print(ast.parse('t.p.func(a, b=c, *d, **e)', mode='eval').body.func.attr) # importatn


print("\n")

print(ast.dump(ast.parse('func(a, b=c, *d, **e)', mode='eval')))
print(ast.parse('func(a, b=c, *d, **e)', mode='eval').body.func.id)
print(ast.parse('func(a, b=c, *d, **e)', mode='eval').body.func.attr)


Expression(body=Str(s='_'))
False
False
t
func
Expression(body=Call(func=Attribute(value=Attribute(value=Name(id='t', ctx=Load()), attr='p', ctx=Load()), attr='func', ctx=Load()), args=[Name(id='a', ctx=Load()), Starred(value=Name(id='d', ctx=Load()), ctx=Load())], keywords=[keyword(arg='b', value=Name(id='c', ctx=Load())), keyword(arg=None, value=Name(id='e', ctx=Load()))]))
t
p
func


Expression(body=Call(func=Name(id='func', ctx=Load()), args=[Name(id='a', ctx=Load()), Starred(value=Name(id='d', ctx=Load()), ctx=Load())], keywords=[keyword(arg='b', value=Name(id='c', ctx=Load())), keyword(arg=None, value=Name(id='e', ctx=Load()))]))
func


AttributeError: 'Name' object has no attribute 'attr'

In [29]:
tree = ast.parse('a.func(a, b=c, *d, **e)', mode='eval')

for child in ast.walk(tree):
     if isinstance(child, ast.Call):
            print(child.func)
        if child.func.id == "func": # child_node.func.attr: this returns the function call's name
            
            print("1")

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 6)

In [59]:
with open("/home/amirmahdi/projects/Adaptive-Logging-system-git/test.py", "r") as file:
    s = file.read()
tree = ast.parse(s)
#print(s)
print(func_call_finderr("Post", tree))
print(func_call_finder("order_by", tree))
print(func_call_finder("start", tree))
print(func_call_finder("join", tree))
print(func_call_finder("Lock", tree))
print(func_call_finder("acquire", tree))
print(func_call_finder("release", tree))

"""print(nested_loop_level_finder(tree))
for c in ast.walk(tree):
    if isinstance(c, ast.For):
        print("y")"""

0
attr:order_by
attr:order_by
attr:order_by
3
start
attr:start
2
attr:join
attr:join
string
2
0
0
0


'print(nested_loop_level_finder(tree))\nfor c in ast.walk(tree):\n    if isinstance(c, ast.For):\n        print("y")'

In [4]:


# Create a sample DataFrame
data = {
    'A': [10, 5, 8],
    'B': [7, 15, 3],
    'C': [12, 9, 6]
}

df = pd.DataFrame(data)

# Find the maximum value in the DataFrame
max_value = df.max().max()

print(max_value)


15


In [12]:
df.iloc[1].sum()

29